# Trabajo Práctico 2: Capa de Transporte
## Redes y Transmisión de Datos, UNPSJB, Sede Trelew, 2018
---
## Cátedra:
   ### * Mg. Ricardo López
   ### * Lic. Nahuel Defossé
---
### Alumno:
   ### * Luciano Serruya Aloisi

# PARTE 1: TCP Fundamentos

## 1. Dos aplicaciones en las máquinas A y B intercambian datos utilizando el protocolo TCP.

### a) Durante el intercambio de datos entre A y B, luego de establecida la conexión, la máquina A no recibe una confirmación de B sobre su ultimo envío, por lo que retransmite el segmento no confirmado. Al cabo de un tiempo, B recibe las dos copias del segmento tal como A las envió. ¿B detecta que se trata de dos copias de la misma información? Justifique su respuesta.

El host B detecta que los segmentos recibidos son idénticos debido a que tiene el mismo *número de secuencia (SEQ)*; se queda con una copia y desecha el otro (*drop*). 

### b) Suponga que la conexión TCP que A inició con B, es una sesión telnet, (o sea que A se conectará al puerto 23 de B). ¿Es posible que A inicie una segunda conexión TCP hacia el puerto 23 de B, manteniendo la primera conexión activa?  Justifique su respuesta e indique, en caso de ser posible, cómo diferenciará A y B los segmentos correspondientes a cada una de las conexiones.

El proceso del host B que esté corriendo telnet, tendrá ocupado el puerto 23; el recurso lo tiene reservado pura y exclusivamente para él. Si el host A se quiere conectar por segunda vez al puerto 23 del host B, no tedrá respuesta ya que estará ocupado con la primer conexión.

Si el host A se logró conectar con el host B, entonces otro puerto estará siendo utilizado por el host B para manetener la conexión.

Los segmentos correspondientes a cada aplicación se identificarán unívocamente con una tupla formada de 4 valores:
* IP del emisor
* Puerto del emisor por el cuál están saliendo los datos 
* IP del destinatario
* Puerto del destinatario que está escuchando la conexión


## 2. Responda: Verdadero o Falso y justifique.
## El uso de ventanas de un protocolo:

### a) Permite aumentar la eficiencia de la transmisión.

Verdadero. Se pueden enviar varios paquetes secuencialmente, sin esperar el reconocimiento por parte del receptor (como sucede con los procotocolos de parada y espera). Cabe aclarar también que en protocolos tipo Go-Back-N, el uso de vetanas deslizantes puede jugar en contra, ya que se envían varios paquetes en simultáneo, pero si el receptor pide una retransmisión, se reenvían esos varios paquetes (generando más carga en la red).

### b) Garantiza el control de errores.

Falso. El protocolo de la capa de transporte (ya sea UDP o TCP) es el encargado de detectar los errores, y no el uso de las ventanas

### c) Requiere la identificación de las tramas.

Verdadero. Se identifican las tramas con un número de secuencia para saber:
* Cuál fue enviado y reconocido
* Cuál fue enviado y no reconocido todavía
* Cuál no fue enviado todavía
* Cuál no fue considerado todavía (están por fuera de la ventana)

### d) Disminuye el overhead del protocolo.

Falso. Justamente al contrario, incorporan un mayor trabajo al protocolo debido a que tienen que actualizar la ventana 

### e) Realiza el control de CRC.

Falso. El Control de Redundancia Cíclica le corresponde a la capa de enlace

### f) No acepta paquetes fuera de orden.

Falso. El protocolo Selective Repeat recibe paquetes fuera de orden y los almacena hasta que llegan los paquetes anteriores (con número de secuencia inferiores). Una vez recibidos todos los paquetes, se los entrega a la capa superior (de aplicación)

### BPF = Berkeley Packet Filter

## 3. Utilice el software Cliente – Servidor de la parte 2 del TP1 y lance el analizador de protocolos para observar la siguiente secuencia:

### a) Establezca conexión entre el cliente y el servidor, enviando un campo de datos del orden de 10 caracteres. Cierre luego la conexión.

Corriendo en una terminal el servidor TCP de ECO con el comando `./bin/echo` y en otra ejecutar el cliente con el comando `echo "0123456789" | ./bin/tcp_client --echo`, se puede obtener con `tcpdump` una captura como la de `pcap/ejercicio3.pcap` 

### b) Indique la cantidad de paquetes circularon entre ambos extremos desde el inicio de la conexión hasta su cierre.

Para toda la comunicación circularon 10 paquetes
* 3 Para el 3-Way handshake
* 4 para el envío de los datos
* 3 más para el cierre de la conexión

### c) Observe e indique las banderas que se observan en el 3-Way y 4-Way de inicio y cierre de la conexión.

* Para el 3-Way handshake el cliente envía la bandera SYN, a lo que el servidor le contesta SYN y ACK. El cliente le envía ACK
* Para el 4-Way handshake (haciendo la captura en un host con una distribución de Linux), se observan 3 paquetes. El primero es del cliente al servidor con las banderas FIN y ACK; el servidor le responde FIN y ACK. Por último el servidor le envía un ACK.

### d) Indique el número de secuencia que se observa en ambos extremos al inicio de la conexión. Y la ventana de Flujo de ambos extremos.

#### SEQ -> Número de secuencia
#### WIN -> Tamaño de ventana

* Primer mensaje del cliente -> SEQ: 1608898858, WIN: 43690 
* Respuesta del servidor -> SEQ: 2170056674, WIN: 43690

### e) Indique los números de secuencia que se observan para ambos extremos al término del 3-Way state (sin envío de datos). Qué le parece particular?

Terminado el 3-Way handshake, los números de secuencia son el ACK delgmento se anterior más el largo del paquete actual

### f) Indique los números de secuencia que se observan para ambos extremos al inicio del cierre de la conexión.

* Primer mensaje de fin de conexión del cliente -> SEQ: 19
* Respuesta del servidor -> SEQ: 35

### g) Indique los números de secuencia que se observan para ambos extremos al término del cierre de la conexión. Qué le parece particular?

El último mensaje enviado para el cierre de la conexión tiene un SEQ=20, siendo 20 el ACK del segmento anterior, lo que significa que ese último segmento tenía un largo de 0.

## 4. Dado el siguiente segmento TCP (se excluye la cabecera IP), enviado por un host A a otro B:

```
04 02 00 50 00 00 32 98
00 00 00 00 60 02 20 00
13 03 00 00 02 04 05 B4
```

04 02 -> puerto fuente

00 50 -> puerto destino (LE ESTÁ PEGANDO AL PUERTO 80 SEÑORE ES UN SERVIDOR WEB)

00 00 32 98 -> SEQ

00 00 00 00 ACK

60 02 cosas

6    0    0    2
0110 xxxx xx00 0010

WIN 6

URG 0 
ACK 0
PSH 0
RST 0
SYN 1
FIN 0

20 00 WIN

13 03 CHECKSUM
00 00 URGENT DATA POINTER

02 04 05 B4 OPTIONS

### a) Qué función cumple el segmento enviado?

El segmento enviado es el primero de un 3-Way handshake para establecer la conexión con el host B ya que solamente tiene la bandera SYN levantada

### b) Caracterice a los Hosts A y B como cliente/servidor.

El Host A es el cliente ya que le está enviando al Host B (servidor) un segmento que solamente tiene levantada la bandera SYN

### d) Cuál será el número de secuencia que utilizará el Host A en el próximo segmento que le envíe a B?

El número de secuencia del próximo segmento que le envíe A a B será un 1

### e) Cuál es el servicio requerido?

El servicio requerido es el de un servidor web (el Host B es un servidor web)

## 5. a. Suponga que la ventana de congestión de TCP está en 18 Kbytes. La ventana publicada por el otro extremo es de 64 Kbyte. ¿A qué valor llegará dicha ventana si los siguientes 5 segmentos transmitidos resultan exitosos y no se recibió aún ningún ACK? Suponga un tamaño máximo de segmento de 2 Kbytes.

Se envían los paquetes y llegan bien, pero no se reciben los ACK -> el transmisor lo toma como que hay congestión en la red y decide reducir su cwnd

Como se vencen los timers del receptor (*timeout*), primero se actualiza el ```sstresh``` con el valor del ```cwnd``` / 2 = 9KBytes. Luego el ```cwnd``` pasa a ser 1 MSS (2 KBytes).

## b. Suponga ahora que luego de transmitir los 5 segmentos exitosos llega un ACK acumulativo reconociendo tres paquetes y que el umbral se encuentra en 12Kbytes.  A qué valor llegará la ventana de congestión.

Con la llegada del ACK, la ventana de congestión pasará a ser ```cwnd``` + ```1 MSS``` = 4 KBytes

## 6. Determine el tamaño óptimo de ventana para una sesión TCP en la que el RTT=100mseg, MSS=600 bytes y velocidad de la interfase 128 Kbps.

## 7. Qué mejora aporta el mecanismo de retransmisión rápida (fast retransmit) de TCP? Indique si aporta mejora, si la empeora o si es indiferente. Justifique.

### A. Permite que no llegue a ocurrir congestión en la red.

me parece que no chee e

### B. Permite que un proceso detecte que hay congestión antes de que esta alcance un valor demasiado alto.

Este puede ser verdadero eh

### C. Permite que la ventana de recepción pueda ser más grande.

eeee me parece que no tiene nada que ver

### D. Aumenta la Eficiencia de la comunicación.

Bueno este sí puede ser porque, osea, obvio no? quee coso

### E. Permite que la congestión ocurra antes.

mhhhh ojo que esteeee eeeeee que la congestion ocurra antes uhmmmmm y puede ser, porque estarías cargando más la red?